<a href="https://colab.research.google.com/github/sraeisi/MachineLearning_Physics/blob/master/Lec_7/Convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import requests
import io

## Farsi Hand-written digits classification

In [0]:
'''
Let's import the Farsi digits (your assignment)
'''

url = 'https://raw.githubusercontent.com/sraeisi/MachineLearning_Physics/master/Assignments/A3/X.npz'
s=requests.get(url, stream=True).content
X = np.load(io.BytesIO(s) )['arr_0']
url = 'https://raw.githubusercontent.com/sraeisi/MachineLearning_Physics/master/Assignments/A3/Y.npz'
s=requests.get(url, stream=True).content
Y = np.load(io.BytesIO(s) )['arr_0']

from sklearn.utils import shuffle
X, Y = shuffle(X,Y, random_state = 0 )

In [4]:
print('Each image is of the form: {}'.format(X[0].shape ) )

Each image is of the form: (32, 32)


In [0]:
n_w, n_h = X[0].shape

In [7]:
# model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(32, 32)),
#     keras.layers.Dense(128, activation=tf.nn.relu),
#     keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(n_w , n_h) ))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 132,490
Trainable params: 132,490
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [9]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,Y_train, 
                    epochs=5, batch_size = 64, 
                   validation_data = (X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 76764 samples, validate on 25588 samples
Epoch 1/5
76764/76764 [==============================] - 6s 83us/step - loss: 0.1518 - acc: 0.9562 - val_loss: 0.0819 - val_acc: 0.9759
Epoch 2/5
76764/76764 [==============================] - 4s 46us/step - loss: 0.0568 - acc: 0.9834 - val_loss: 0.0640 - val_acc: 0.9805
Epoch 3/5
76764/76764 [==============================] - 4s 46us/step - loss: 0.0359 - acc: 0.9895 - val_loss: 0.0560 - val_acc: 0.9825
Epoch 4/5
76764/76764 [==============================] - 4s 50us/step - loss: 0.0256 - acc: 0.9919 - val_loss: 0.0499 - val_acc: 0.9855
Epoch 5/5
76764/76764 [==============================] - 4s 52us/step - loss: 0.0174 - acc: 0.9947 - val_loss: 0.0531 - val_acc: 0.9845


In [10]:
Y_predicted = np.argmax( model.predict(X_test) , axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2522    0    0    0    0   12    1    1    0    0]
 [   1 2505    0    0    1    0    4    0    0    0]
 [   2    8 2425   37    9    0   16    2    0    1]
 [   0    0   37 2465   39    1    2    0    0    0]
 [   3    4   12   29 2615   12   10    5    2    0]
 [   4    1    1    1    1 2481    3    1    4    0]
 [   1    2    1    0    1    1 2590    3    0    9]
 [   2    3    4    0    3    3    9 2584    0    0]
 [   0    5    0    1    1    1    5    1 2526    8]
 [   0    9    0    0    2    0   48    0    7 2478]]


## Convnet

In [0]:
X_train[0].shape

(32, 32)

In [11]:
n_w , n_h = X[0].shape
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(n_w , n_h, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
__________

In [13]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train = X_train.reshape(-1, n_w, n_h, 1)
X_test = X_test.reshape(-1, n_w, n_h, 1)

history = model.fit(X_train,Y_train, 
                    epochs=5, batch_size = 64, 
                   validation_data = (X_test, Y_test))

Train on 76764 samples, validate on 25588 samples
Epoch 1/5
76764/76764 [==============================] - 9s 117us/step - loss: 0.1501 - acc: 0.9534 - val_loss: 0.0490 - val_acc: 0.9844
Epoch 2/5
76764/76764 [==============================] - 6s 76us/step - loss: 0.0348 - acc: 0.9888 - val_loss: 0.0296 - val_acc: 0.9905
Epoch 3/5
76764/76764 [==============================] - 6s 80us/step - loss: 0.0244 - acc: 0.9923 - val_loss: 0.0313 - val_acc: 0.9898
Epoch 4/5
76764/76764 [==============================] - 6s 77us/step - loss: 0.0180 - acc: 0.9942 - val_loss: 0.0197 - val_acc: 0.9944
Epoch 5/5
76764/76764 [==============================] - 6s 74us/step - loss: 0.0144 - acc: 0.9952 - val_loss: 0.0171 - val_acc: 0.9947


In [14]:
Y_predicted = np.argmax( model.predict(X_test ), axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2519    1    0    0    0   15    0    1    0    0]
 [   0 2509    2    0    0    0    0    0    0    0]
 [   1    1 2492    2    3    0    1    0    0    0]
 [   0    0   27 2503   12    1    1    0    0    0]
 [   0    0   10    6 2670    1    5    0    0    0]
 [   2    0    0    0    0 2492    3    0    0    0]
 [   0    1    0    0    1    1 2597    1    0    7]
 [   0    1    3    0    1    0    1 2602    0    0]
 [   0    4    0    0    0    0    0    0 2544    0]
 [   0    5    0    0    1    0   12    0    1 2525]]


In [0]:
'''Implement LeNet-5'''